In [ ]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Prompt Pipelining Overview

Prompt pipelining is a powerful tool designed for those who seek a modular and efficient approach to prompt design.


## Here are some common cases where prompt pipelining can be useful:

 - Reusing prompt components like introductions, instructions, examples, etc. across multiple prompts. Prompt pipelining allows easily reusing these modular blocks.

 - Splitting up a long prompt into smaller logical chunks. This can make prompts more readable and maintainable.

 - Dynamically assembling prompts based on conditionals or other logic. You can build prompts on the fly from reusable parts.

 - Creating prompts in a loop by appending to an existing base prompt. Useful for things like few-shot learning prompts.

 - Mixing static text with templates containing variables. The static text can provide structure while the templates inject dynamic content.

 - Composing chat prompts from message templates and static messages. Each piece gets appended as a new message.

 - Building up prompts from user provided components for customization. Prompt pipelining allows prompts to be assembled from modular parts.

 - Any situation where you want to build up prompts in a reusable composable way, prompt pipelining provides a clean interface for that.

## String Prompt Pipelining

For string prompts, templates are interconnected in sequence.

You have the liberty to utilize either direct prompts or strings.

However, it's essential to note that the initial element in the sequence should always be a prompt.

In [ ]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("I'm heading to {destination}. ")
    + "Recommend a great {activity} spot!")

prompt


In [ ]:
prompt = prompt + "\n\nAlso, any local delicacies I should try?"
prompt

In [ ]:
prompt.format(destination="Punjab", activity="dining")

### The key differences between prompt pipelining and multi-input prompts are:

 - Prompt pipelining composes multiple prompts/strings into a single final prompt. Multi-input prompts allow passing multiple inputs to a single prompt template.

 - With pipelining, each component can be formatted independently before joining. Multi-input just formats a single template.

 - Pipelining joins components into a linear sequence. Multi-input prompts are a single template handling multiple inputs.

 - Pipelining allows reuse of components like introductions, examples, etc. Multi-input is focused on handling multiple query inputs.

 - Pipelining output is a single prompt. Multi-input output is the completion based on multiple inputs.

 - Pipelining lets you build up prompts modularly. Multi-input is one template handling multiple inputs.

In summary, prompt pipelining composes multiple prompt components into one final prompt in a linear sequence. Multi-input prompts allow a single template to handle multiple input variables in parallel. Pipelining focuses on modular prompt building, while multi-input handles multiple query inputs.

# Use in a chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"destination":"Punjab", "activity":"dining"})

In [ ]:
prompt = prompt +" How should I greet the locals in a jolly, informal manner?"

chain = prompt | llm | output_parser

chain.invoke({"destination":"Punjab", "activity":"dining"})

# Example usecase

In [ ]:
class TravelChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI(model="gpt-3.5-turbo-1106")
        self.base_prompt = PromptTemplate.from_template(base_template)

    def append_to_prompt(self, additional_text):
        self.base_prompt += additional_text

    def run_chain(self, destination, activity):
        output_parser = StrOutputParser()
        chain = self.base_prompt | self.model | output_parser
        return chain.invoke({"destination":destination, "activity":activity})

# Usage
base_template = "I'm heading to {destination}. Recommend a great {activity} spot!"

chatbot = TravelChatbot(base_template)
# Basic prompt
chatbot.run_chain(destination="Punjab", activity="dining")

In [ ]:
# Append more to the prompt and run again
chatbot.append_to_prompt("\n\nAlso, any local delicacies I should try?")
print(chatbot.run_chain(destination="Punjab", activity="dining"))

In [ ]:
chatbot.append_to_prompt(" How should I greet the locals in a friendly, informal, jolly colloquial manner?")
chatbot.run_chain(destination="Punjab", activity="dining")

# Chat Prompt Pipeline

A chat prompt is made up a of a list of messages. Purely for developer experience, we've added a convinient way to create these prompts. In this pipeline, each new element is a new message in the final prompt.

### Key Insights into Chat Prompt Pipelining in Langchain:

Composition: Chat prompt pipelining facilitates the creation of chat prompts using reusable message components. Each addition to the pipeline translates to a new message in the final chat prompt.

Versatility: You can seamlessly integrate static message objects, such as HumanMessage and AIMessage, with variable-containing message templates.

End Result: The culmination is a unified ChatPromptTemplate, primed for formatting and integration into a chain. This structure promotes the effortless reuse of components, including instructions and examples.

Modularity: This approach champions the dynamic construction of chat prompts using modular blocks, ensuring flexibility and efficiency.

So in summary, chat prompt pipelining composes chat prompts from reusable message templates and static messages. It allows dynamically constructing prompts from logical blocks in a user friendly way. The end result is a single reusable ChatPromptTemplate.



In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Setting the scene with a Cockney-themed system message
prompt = SystemMessage(content="Welcome to the East End Cockney Chat! 🇬🇧")

# Constructing a chat flow with dry humour
new_prompt = (
    prompt
    + HumanMessage(content="Alright, guv'nor?")
    + AIMessage(content="Not too shabby. Did you hear about the London fog?")
    + "{input}"
)

# Formatting the chat with the user's response
new_prompt.format_messages(input="No, what about it?")
print(new_prompt)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

model = ChatOpenAI()

chain = LLMChain(llm=model, prompt=new_prompt)

# Running the chatbot to get the punchline
response = chain.run("No, what about it?")
response

# Prompt Composition

This can be useful when you want to reuse parts of prompts. This can be done with a PipelinePrompt. A PipelinePrompt consists of two main parts:

1. Final prompt: The final prompt that is returned

2. Pipeline prompts: A list of tuples, consisting of a string name and a prompt template. Each prompt template will be formatted and then passed to future prompt templates as a variable with the same name.

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [ ]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [ ]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [ ]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [ ]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [ ]:
pipeline_prompt.input_variables

In [ ]:
last_prompt = pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Tesla",
    input="What's your favorite social media site?"
)

print(last_prompt)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

llm.invoke(last_prompt)

In [ ]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

class CookingShowChatbot:
    def __init__(self):
        # Base template for the cooking show scenario
        self.full_template = """{introduction}
        {example_dish}

        {present_dish}"""
        self.full_prompt = PromptTemplate.from_template(self.full_template)

        # Introduction where the user impersonates a famous chef
        self.introduction_template = """Welcome to the cooking show! Today, you're channeling the spirit of Chef {chef_name}."""
        self.introduction_prompt = PromptTemplate.from_template(self.introduction_template)

        # Example dish made by the famous chef
        self.example_dish_template = """Remember when Chef {chef_name} made that delicious {example_dish_name}? It was a hit!"""
        self.example_dish_prompt = PromptTemplate.from_template(self.example_dish_template)

        # User's turn to present their dish
        self.present_dish_template = """Now, it's your turn! Show us how you make your {user_dish_name}. Let's get cooking!"""
        self.present_dish_prompt = PromptTemplate.from_template(self.present_dish_template)

        # Combining the prompts into a pipeline
        self.input_prompts = [
            ("introduction", self.introduction_prompt),
            ("example_dish", self.example_dish_prompt),
            ("present_dish", self.present_dish_prompt)
        ]
        self.pipeline_prompt = PipelinePromptTemplate(final_prompt=self.full_prompt,
                                                      pipeline_prompts=self.input_prompts
                                                      )

    def run_scenario(self, chef_name, example_dish_name, user_dish_name):
        return self.pipeline_prompt.format(
            chef_name=chef_name,
            example_dish_name=example_dish_name,
            user_dish_name=user_dish_name
        )

# Usage
chatbot = CookingShowChatbot()
scenario = chatbot.run_scenario(chef_name="Gordon Ramsay", example_dish_name="Beef Wellington", user_dish_name="Vegan Lasagna")
print(scenario)

In [ ]:
response = llm.invoke(scenario)

print(response.content)